## Data Scraping
First step would to be grab the necessary data and save it in our appropriate medium.
In this project I'll be building a RAG for vehicle purchasing and therefore the data I'll need to hold is vehicle statistics.

https://selenium-python.readthedocs.io/

https://www.auto-data.net/en/

In [1]:
# auto install selenium if none present
%pip install selenium
%pip install mysql-connector-python



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import csv


In [16]:
driver = webdriver.Chrome()
driver.get("https://www.auto-data.net/en/")

driver.implicitly_wait(0.5)
manufacturers = driver.find_element(By.CLASS_NAME, "markite") 
manufacturers_links = []

# Open the CSV file in append mode to write data as it loops
with open('manufacturers_links.csv', mode='a', newline='') as file:
    writer = csv.writer(file)
    for manufacturer in manufacturers.find_elements(By.TAG_NAME, "a"):
        manufacturer_name = manufacturer.text
        link = manufacturer.get_attribute("href")
        manufacturers_links.append((manufacturer_name, link))
        writer.writerow([manufacturer_name, link])  # Write the data row by row

print(manufacturers_links)
driver.quit()

[('Acura', 'https://www.auto-data.net/en/acura-brand-6'), ('Alfa Romeo', 'https://www.auto-data.net/en/alfa-romeo-brand-11'), ('Alpina', 'https://www.auto-data.net/en/alpina-brand-16'), ('Aston Martin', 'https://www.auto-data.net/en/aston-martin-brand-36'), ('Audi', 'https://www.auto-data.net/en/audi-brand-41'), ('Bentley', 'https://www.auto-data.net/en/bentley-brand-66'), ('BMW', 'https://www.auto-data.net/en/bmw-brand-86'), ('Bugatti', 'https://www.auto-data.net/en/bugatti-brand-106'), ('BYD', 'https://www.auto-data.net/en/byd-brand-116'), ('Cadillac', 'https://www.auto-data.net/en/cadillac-brand-121'), ('Chevrolet', 'https://www.auto-data.net/en/chevrolet-brand-156'), ('Chrysler', 'https://www.auto-data.net/en/chrysler-brand-161'), ('Citroen', 'https://www.auto-data.net/en/citroen-brand-166'), ('Cupra', 'https://www.auto-data.net/en/cupra-brand-256'), ('Dacia', 'https://www.auto-data.net/en/dacia-brand-181'), ('Daewoo', 'https://www.auto-data.net/en/daewoo-brand-191'), ('Daihatsu', 

In [ ]:
with open('manufacturers_links.csv', mode='r') as file:
    reader = csv.reader(file)
    for name, link in reader:
        
        if(name.lower() == "All brands".lower()): continue
        with open('manufacturers_models.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            driver.get(link)
            try:
                models = driver.find_element(By.CLASS_NAME , "modelite")
                for model in models.find_elements(By.CLASS_NAME, "modeli"):
                    model_name = model.text
                    model_link = model.get_attribute("href")
                    writer.writerow([name, model_name, model_link])
            except:
                print("No models found for ", name)
        
        driver.quit()


In [5]:
driver = webdriver.Chrome()
driver.implicitly_wait(0.5)
with open('manufacturers_models.csv', mode='r') as file:
    reader = csv.reader(file)
    for name, model, link in reader:
        with open('manufacturer_variants.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            driver = webdriver.Chrome()
            driver.implicitly_wait(0.5)
            driver.get(link)
            try:
                for row in driver.find_elements(By.TAG_NAME, "tr"):
                    try:
                        link = row.find_element(By.XPATH, ".//th/a").get_attribute("href")
                        writer.writerow([name, model, link])
                    except Exception as e:
                        print(f"Could not extract link for a variant")
            except:
                print("No variants found for ", name, model)
            driver.quit()


SyntaxError: invalid syntax (934658058.py, line 2)

In [ ]:

driver = webdriver.Chrome()
driver.implicitly_wait(0.5)
with open('manufacturer_variants.csv', mode='r') as file:
    reader = csv.reader(file)
    for manufacturer_name, model_name, link in reader:
        with open('manufacturer_specific_variants.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            driver.get(link)
            variants = driver.find_elements(By.TAG_NAME, "tr")
            for variant in variants:
                try:
                    inner_element = variant.find_element(By.XPATH, ".//th/a")  # Relative XPath
                    href = inner_element.get_attribute("href")
                    writer.writerow([manufacturer_name, model_name, href])
                except Exception as e:
                    print(f"Could not extract link for a specific variant")
driver.quit()             
# stopped at skoda superb    

In [27]:
import mysql.connector

HOST = "localhost"
USER = "root"
PASSWORD = ""
DATABASE = "vehicle_data"

# Connect to MySQL DB
connection = mysql.connector.connect(
    host=HOST,
    user=USER,
    password=PASSWORD
)
# Create a cursor object
cursor = connection.cursor()

# Create the database if it doesn't exist
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {DATABASE}")

# Select the database
cursor.execute(f"USE {DATABASE}")


In [28]:
driver = webdriver.Chrome()
driver.implicitly_wait(1)
with open('manufacturer_specific_variants.csv', mode='r') as file:
    reader = csv.reader(file)
    for model, variant, link in reader:
        try:
            driver.get(link)
            tableRows = driver.find_element(By.CLASS_NAME, "cardetailsout").find_element(By.TAG_NAME, "tbody").find_elements(By.TAG_NAME, "tr")
            vehicle = {}
            for row in tableRows:
                try:
                    # Extract key (th) and value (td) from the current row
                    key = row.find_element(By.TAG_NAME, "th").text
                    value = row.find_element(By.TAG_NAME, "td").text
                    vehicle[key] = value
                except Exception as e:
                    # print(f"Could not extract key-value pair from row: {e}")
                    continue
            print(vehicle)
            
            for key in vehicle.keys():
                # Check if the column exists
                cursor.execute(f"SHOW COLUMNS FROM vehicles LIKE '{key}'")
                result = cursor.fetchone()
                
                if result is None:  # If the column does not exist, add it
                    alter_table_query = f"ALTER TABLE vehicles ADD COLUMN `{key}` VARCHAR(255)"
                    cursor.execute(alter_table_query)
                
                # Prepare the INSERT statement
                columns = ", ".join(f"`{key}`" for key in vehicle.keys())
                placeholders = ", ".join(["%s"] * len(vehicle))
                values = tuple(vehicle.values())

                # Insert query
                insert_query = f"INSERT INTO vehicles ({columns}) VALUES ({placeholders})"
                print(f"MY QUERY IS:\n{insert_query}")
                cursor.execute(insert_query, values)
                connection.commit()
                
        except Exception as e:
            print(f"Could not find table rows on page: {e}")

driver.quit()

{'Brand': 'Acura', 'Model': 'CL', 'Generation': 'CL II', 'Modification (Engine)': '3.2 i V6 24V Type S (263 Hp) Automatic', 'Start of production': '2000 year', 'End of production': '2003 year', 'Powertrain Architecture': 'Internal Combustion engine', 'Body type': 'Coupe', 'Seats': '5', 'Doors': '2', 'Fuel consumption (economy) - urban': '12.4 l/100 km\n18.97 US mpg\n22.78 UK mpg\n8.06 km/l', 'Fuel consumption (economy) - extra urban': '8.1 l/100 km\n29.04 US mpg\n34.87 UK mpg\n12.35 km/l', 'Fuel Type': 'Petrol (Gasoline)', 'Acceleration 0 - 100 km/h': '6.8 sec', 'Acceleration 0 - 62 mph': '6.8 sec', 'Acceleration 0 - 60 mph (Calculated by Auto-Data.net)': '6.5 sec', 'Maximum speed': '235 km/h\n146.02 mph', 'Weight-to-power ratio': '6.1 kg/Hp, 162.8 Hp/tonne', 'Weight-to-torque ratio': '5.1 kg/Nm, 194.4 Nm/tonne', 'Power': '263 Hp @ 6100 rpm.', 'Power per litre': '81.9 Hp/l', 'Torque': '314 Nm @ 3500 rpm.\n231.59 lb.-ft. @ 3500 rpm.', 'Engine layout': 'Front, Transverse', 'Engine Model/

KeyboardInterrupt: 

In [11]:
import mysql.connector

HOST = "localhost"
USER = "root"
PASSWORD = ""
DATABASE = "vehicle_data"

# Connect to MySQL DB
connection = mysql.connector.connect(
    host=HOST,
    user=USER,
    password=PASSWORD
)
# Create a cursor object
cursor = connection.cursor()

# Create the database if it doesn't exist
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {DATABASE}")
# Select the database
cursor.execute(f"USE {DATABASE}")

# Create the table if it doesn't exist
create_table_query = """
CREATE TABLE IF NOT EXISTS vehicles (
    id INT AUTO_INCREMENT PRIMARY KEY
)
"""
cursor.execute(create_table_query)

# Sample vehicle data for testing
vehicles = [
    {
        "Engine": "V6",  # Removed parentheses from column name
        "Start_of_production": "2010",  # Changed spaces to underscores
        "End_of_production": "2015",
        "Powertrain_Architecture": "AWD",
        "Body_Style": "SUV"
    },
    {
        "Engine": "V8",
        "Start_of_production": "2016",
        "End_of_production": "2020",
        "Powertrain_Architecture": "RWD",
        "Body_Style": "Sedan"
    }
]

for vehicle in vehicles:
    # Check and create columns if they don't exist
    for key in vehicle.keys():
        # Check if the column exists
        cursor.execute(f"SHOW COLUMNS FROM vehicles LIKE '{key}'")
        result = cursor.fetchone()
        
        if result is None:  # If the column does not exist, add it
            alter_table_query = f"ALTER TABLE vehicles ADD COLUMN `{key}` VARCHAR(255)"
            cursor.execute(alter_table_query)

    # Prepare the INSERT statement
    columns = ", ".join(f"`{key}`" for key in vehicle.keys())
    placeholders = ", ".join(["%s"] * len(vehicle))
    values = tuple(vehicle.values())

    # Insert query
    insert_query = f"INSERT INTO vehicles ({columns}) VALUES ({placeholders})"
    cursor.execute(insert_query, values)

# Commit the changes
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()

ProgrammingError: 1054 (42S22): Unknown column 'Start_of_production' in 'field list'